In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
import seaborn as sns
import pickle

In [2]:
train = pd.read_csv('data/train.csv', index_col = 'no')
test = pd.read_csv('data/test.csv', index_col = 'no')

In [3]:
train.income

no
1        0
2        0
3        0
4        0
5        1
        ..
29301    0
29302    0
29303    0
29304    0
29305    0
Name: income, Length: 29305, dtype: int64

In [4]:
targets = train.income  # 정답
train.drop(['income'], axis = 1, inplace=True)  # 일단 지워주기

In [5]:
combined = train.append(test,ignore_index=True)  # 테스트와 트레인 합치기

In [6]:
combined

age          workclass  fnlwgt      education  education-num  \
0       25            Private  219199           11th              7   
1       39            Private   52978   Some-college             10   
2       35            Private  196899      Bachelors             13   
3       64            Private  135527      Assoc-voc             11   
4       24            Private   60783   Some-college             10   
...    ...                ...     ...            ...            ...   
48837   45   Self-emp-not-inc  116789        HS-grad              9   
48838   48            Private  185079        HS-grad              9   
48839   63            Private  117473   Some-college             10   
48840   18            Private  150817           11th              7   
48841   31            Private  341632     Assoc-acdm             12   

            marital-status          occupation    relationship  \
0                 Divorced   Machine-op-inspct   Not-in-family   
1                 Divorced       Other-service   Not-in-family   
2            Never-married   Handlers-cleaners   Not-in-family   
3                 Divorced        Tech-support   Not-in-family   
4       Married-civ-spouse    Transport-moving         Husband   
...                    ...                 ...             ...   
48837   Married-civ-spouse        Craft-repair         Husband   
48838        Never-married     Exec-managerial   Not-in-family   
48839   Married-civ-spouse      Prof-specialty         Husband   
48840        Never-married               Sales       Own-child   
48841   Married-civ-spouse        Tech-support         Husband   

                      race      sex  capital-gain  capital-loss  \
0                    White     Male             0             0   
1                    White   Female             0          1721   
2       Asian-Pac-Islander   Female             0             0   
3                    White   Female             0             0   
4                    White     Male             0             0   
...                    ...      ...           ...           ...   
48837                White     Male             0             0   
48838                White   Female             0             0   
48839                White     Male          4386             0   
48840                White   Female             0             0   
48841                Black     Male             0             0   

       hours-per-week  native-country  
0                  40   United-States  
1                  55   United-States  
2                  50           Haiti  
3                  40   United-States  
4                  70   United-States  
...               ...             ...  
48837              60   United-States  
48838              50   United-States  
48839              40   United-States  
48840              20   United-States  
48841              46   United-States  

[48842 rows x 14 columns]

In [7]:
combined.shape

(48842, 14)

In [8]:
combined.head(20)

age          workclass  fnlwgt      education  education-num  \
0    25            Private  219199           11th              7   
1    39            Private   52978   Some-college             10   
2    35            Private  196899      Bachelors             13   
3    64            Private  135527      Assoc-voc             11   
4    24            Private   60783   Some-college             10   
5    66   Self-emp-not-inc  274451            9th              5   
6    56            Private  152874      Bachelors             13   
7    70   Self-emp-not-inc  165586        HS-grad              9   
8    21            Private  211013      Assoc-voc             11   
9    33            Private  192644        HS-grad              9   
10   45            Private  174794   Some-college             10   
11   54            Private  231004   Some-college             10   
12   63          Local-gov  241404   Some-college             10   
13   58          Local-gov   36091        Masters             14   
14   44   Self-emp-not-inc   53956   Some-college             10   
15   31            Private  162312      Bachelors             13   
16   54            Private  299324        5th-6th              3   
17   17            Private   52012           11th              7   
18   43            Private  208613    Prof-school             15   
19   18                  ?  220168   Some-college             10   

            marital-status          occupation     relationship  \
0                 Divorced   Machine-op-inspct    Not-in-family   
1                 Divorced       Other-service    Not-in-family   
2            Never-married   Handlers-cleaners    Not-in-family   
3                 Divorced        Tech-support    Not-in-family   
4       Married-civ-spouse    Transport-moving          Husband   
5       Married-civ-spouse     Farming-fishing          Husband   
6                 Divorced      Prof-specialty        Unmarried   
7       Married-civ-spouse     Farming-fishing          Husband   
8       Married-civ-spouse       Other-service   Other-relative   
9                Separated   Handlers-cleaners        Unmarried   
10                Divorced     Priv-house-serv    Not-in-family   
11                Divorced        Adm-clerical    Not-in-family   
12                Divorced     Protective-serv    Not-in-family   
13           Never-married     Exec-managerial    Not-in-family   
14      Married-civ-spouse     Farming-fishing          Husband   
15           Never-married      Prof-specialty    Not-in-family   
16   Married-spouse-absent   Machine-op-inspct        Unmarried   
17           Never-married       Other-service        Own-child   
18   Married-spouse-absent      Prof-specialty    Not-in-family   
19           Never-married                   ?        Own-child   

                   race      sex  capital-gain  capital-loss  hours-per-week  \
0                 White     Male             0             0              40   
1                 White   Female             0          1721              55   
2    Asian-Pac-Islander   Female             0             0              50   
3                 White   Female             0             0              40   
4                 White     Male             0             0              70   
5                 White     Male             0             0              25   
6                 Black   Female             0          1741              40   
7                 White     Male             0             0              40   
8                 White   Female             0             0              50   
9                 White     Male             0             0              35   
10                White   Female             0             0              29   
11                White   Female             0             0              37   
12                White   Female             0             0              35   
13   Amer-Indian-Eskimo   Female             0 

## workclass, occupation 에 있는 ' ?' 값 대체하기

In [9]:
combined['workclass'].value_counts()   # ? 를 Private로 바꿔보자!

 Private             33906
 Self-emp-not-inc     3862
 Local-gov            3136
 ?                    2799
 State-gov            1981
 Self-emp-inc         1695
 Federal-gov          1432
 Without-pay            21
 Never-worked           10
Name: workclass, dtype: int64

In [10]:
# combined[combined['workclass'] == '?']

In [11]:
combined['workclass'][combined['workclass']==' ?']=combined['workclass'][combined['workclass']==' ?'].replace(' ?',' Private')

<ipython-input-11-9037ed781309>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  combined['workclass'][combined['workclass']==' ?']=combined['workclass'][combined['workclass']==' ?'].replace(' ?',' Private')


In [12]:
combined['workclass'].unique()

array([' Private', ' Self-emp-not-inc', ' Local-gov', ' State-gov',
       ' Self-emp-inc', ' Federal-gov', ' Without-pay', ' Never-worked'],
      dtype=object)

In [13]:
combined['occupation'].value_counts() 

 Prof-specialty       6172
 Craft-repair         6112
 Exec-managerial      6086
 Adm-clerical         5611
 Sales                5504
 Other-service        4923
 Machine-op-inspct    3022
 ?                    2809
 Transport-moving     2355
 Handlers-cleaners    2072
 Farming-fishing      1490
 Tech-support         1446
 Protective-serv       983
 Priv-house-serv       242
 Armed-Forces           15
Name: occupation, dtype: int64

In [14]:
combined['native-country'][combined['native-country']==' ?']=combined['native-country'][combined['native-country']==' ?'].replace(' ?',' United-States')
combined['native-country'].value_counts()

<ipython-input-14-24f1b8c91222>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  combined['native-country'][combined['native-country']==' ?']=combined['native-country'][combined['native-country']==' ?'].replace(' ?',' United-States')


 United-States                 44689
 Mexico                          951
 Philippines                     295
 Germany                         206
 Puerto-Rico                     184
 Canada                          182
 El-Salvador                     155
 India                           151
 Cuba                            138
 England                         127
 China                           122
 South                           115
 Jamaica                         106
 Italy                           105
 Dominican-Republic              103
 Japan                            92
 Guatemala                        88
 Poland                           87
 Vietnam                          86
 Columbia                         85
 Haiti                            75
 Portugal                         67
 Taiwan                           65
 Iran                             59
 Nicaragua                        49
 Greece                           49
 Peru                             46
 

In [15]:
#train[['sex', 'occupation', 'hours-per-week']]

In [16]:
#train[['sex', 'occupation', 'hours-per-week']].groupby(['occupation','sex']).mean()

In [17]:
combined['occupation'][combined['occupation']==' ?']=combined['occupation'][combined['occupation']==' ?'].replace(' ?','O')

<ipython-input-17-d6a8fed2918b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  combined['occupation'][combined['occupation']==' ?']=combined['occupation'][combined['occupation']==' ?'].replace(' ?','O')


In [18]:
combined[combined['occupation']=='O']['age'].mean()

40.0690637237451

In [19]:
combined[combined['occupation']=='O']['age'].head(30)

19     18
20     23
24     19
37     24
47     23
53     66
62     23
70     50
94     61
133    19
135    64
140    47
147    64
152    60
153    19
159    60
203    18
213    33
218    73
227    20
239    54
256    19
258    36
278    45
283    76
296    47
337    18
343    66
347    17
368    64
Name: age, dtype: int64

In [20]:
combined[combined['occupation']=='O'].tail(30)

age workclass  fnlwgt      education  education-num  \
48402   18   Private   39493   Some-college             10   
48405   72   Private  272425        HS-grad              9   
48411   18   Private  149343        HS-grad              9   
48421   18   Private  395567           11th              7   
48422   38   Private  212048    Prof-school             15   
48434   19   Private  174233   Some-college             10   
48436   21   Private  138575        HS-grad              9   
48458   37   Private   87369            9th              5   
48495   18   Private  202516        HS-grad              9   
48512   26   Private  408417   Some-college             10   
48532   21   Private  155697            9th              5   
48565   31   Private  233371        HS-grad              9   
48591   32   Private  335427      Bachelors             13   
48594   42   Private  184018   Some-college             10   
48599   19   Private   57329   Some-college             10   
48616   65   Private  231604        HS-grad              9   
48638   17   Private  198797           11th              7   
48677   19   Private   28455   Some-college             10   
48707   18   Private  271935           11th              7   
48709   20   Private  285208   Some-college             10   
48730   37   Private  148266    Prof-school             15   
48738   21   Private  123727   Some-college             10   
48748   23   Private   38455   Some-college             10   
48754   70   Private  118630      Assoc-voc             11   
48769   22   Private  154235   Some-college             10   
48773   24   Private  390608        HS-grad              9   
48779   30   Private  108464   Some-college             10   
48789   31   Private  317761      Bachelors             13   
48804   18   Private   51574        HS-grad              9   
48830   22   Private  244771           11th              7   

            marital-status occupation     relationship                 race  \
48402        Never-married          O        Own-child                White   
48405   Married-civ-spouse          O          Husband                White   
48411        Never-married          O        Own-child                White   
48421        Never-married          O        Own-child                White   
48422             Divorced          O    Not-in-family                White   
48434        Never-married          O        Own-child                Black   
48436        Never-married          O   Other-relative                White   
48458             Divorced          O        Unmarried                White   
48495        Never-married          O        Own-child                White   
48512    Married-AF-spouse          O          Husband                Black   
48532        Never-married          O        Own-child                White   
48565   Married-civ-spouse          O             Wife                Black   
48591   Married-civ-spouse          O             Wife                White   
48594             Divorced          O        Unmarried                White   
48599        Never-married          O        Own-child                White   
48616   Married-civ-spouse          O          Husband                White   
48638        Never-married          O        Own-child                White   
48677        Never-married          O        Own-child                White   
48707        Never-married          O   Other-relative                White   
48709        Never-married          O        Own-child                White   
48730   Married-civ-spouse          O          Husband                White   
48738        Never-married          O    Not-in-family                White   
48748        Never-married          O    Not-in-family                White   
48754              Widowed          O        Unmarried                White   
48769   Married-civ-spouse          O             Wife                White   
48773        Never-married    

In [21]:
combined[(combined['occupation']=='O') & (combined['workclass']==' Never-worked')]

age      workclass  fnlwgt      education  education-num  \
4059    17   Never-worked  131593           11th              7   
9898    17   Never-worked  237272           10th              6   
11467   18   Never-worked  162908           11th              7   
18422   18   Never-worked  157131           11th              7   
22247   23   Never-worked  188535        7th-8th              4   
30740   18   Never-worked  153663   Some-college             10   
33348   30   Never-worked  176673        HS-grad              9   
39169   18   Never-worked  206359           10th              6   
41168   20   Never-worked  462294   Some-college             10   
46999   20   Never-worked  273905        HS-grad              9   

               marital-status occupation     relationship    race      sex  \
4059            Never-married          O        Own-child   Black   Female   
9898            Never-married          O        Own-child   White     Male   
11467           Never-married          O        Own-child   White     Male   
18422           Never-married          O        Own-child   White   Female   
22247                Divorced          O    Not-in-family   White     Male   
30740           Never-married          O        Own-child   White     Male   
33348      Married-civ-spouse          O             Wife   Black   Female   
39169           Never-married          O        Own-child   White     Male   
41168           Never-married          O        Own-child   Black     Male   
46999   Married-spouse-absent          O   Other-relative   White     Male   

       capital-gain  capital-loss  hours-per-week  native-country  
4059              0             0              20   United-States  
9898              0             0              30   United-States  
11467             0             0              35   United-States  
18422             0             0              10   United-States  
22247             0             0              35   United-States  
30740             0             0               4   United-States  
33348             0             0              40   United-States  
39169             0             0              40   United-States  
41168             0             0              40   United-States  
46999             0             0              35   United-States

In [22]:
combined[combined['workclass']==' Never-worked']

age      workclass  fnlwgt      education  education-num  \
4059    17   Never-worked  131593           11th              7   
9898    17   Never-worked  237272           10th              6   
11467   18   Never-worked  162908           11th              7   
18422   18   Never-worked  157131           11th              7   
22247   23   Never-worked  188535        7th-8th              4   
30740   18   Never-worked  153663   Some-college             10   
33348   30   Never-worked  176673        HS-grad              9   
39169   18   Never-worked  206359           10th              6   
41168   20   Never-worked  462294   Some-college             10   
46999   20   Never-worked  273905        HS-grad              9   

               marital-status occupation     relationship    race      sex  \
4059            Never-married          O        Own-child   Black   Female   
9898            Never-married          O        Own-child   White     Male   
11467           Never-married          O        Own-child   White     Male   
18422           Never-married          O        Own-child   White   Female   
22247                Divorced          O    Not-in-family   White     Male   
30740           Never-married          O        Own-child   White     Male   
33348      Married-civ-spouse          O             Wife   Black   Female   
39169           Never-married          O        Own-child   White     Male   
41168           Never-married          O        Own-child   Black     Male   
46999   Married-spouse-absent          O   Other-relative   White     Male   

       capital-gain  capital-loss  hours-per-week  native-country  
4059              0             0              20   United-States  
9898              0             0              30   United-States  
11467             0             0              35   United-States  
18422             0             0              10   United-States  
22247             0             0              35   United-States  
30740             0             0               4   United-States  
33348             0             0              40   United-States  
39169             0             0              40   United-States  
41168             0             0              40   United-States  
46999             0             0              35   United-States

In [23]:
combined[(combined['occupation']=='O') & (combined['workclass']==' Private')]

age workclass  fnlwgt      education  education-num  \
19      18   Private  220168   Some-college             10   
20      23   Private  194096   Some-college             10   
24      19   Private   50626   Some-college             10   
37      24   Private  152719   Some-college             10   
47      23   Private   62507   Some-college             10   
...    ...       ...     ...            ...            ...   
48773   24   Private  390608        HS-grad              9   
48779   30   Private  108464   Some-college             10   
48789   31   Private  317761      Bachelors             13   
48804   18   Private   51574        HS-grad              9   
48830   22   Private  244771           11th              7   

            marital-status occupation    relationship                 race  \
19           Never-married          O       Own-child                White   
20           Never-married          O       Own-child                White   
24           Never-married          O       Own-child                Black   
37           Never-married          O       Own-child                Black   
47           Never-married          O   Not-in-family                White   
...                    ...        ...             ...                  ...   
48773        Never-married          O   Not-in-family                White   
48779   Married-civ-spouse          O         Husband                White   
48789        Never-married          O   Not-in-family                White   
48804        Never-married          O       Own-child   Asian-Pac-Islander   
48830            Separated          O       Unmarried                Black   

           sex  capital-gain  capital-loss  hours-per-week  native-country  
19        Male             0             0              16   United-States  
20      Female             0             0              30   United-States  
24      Female             0             0              20   United-States  
37      Female             0             0              15           Haiti  
47      Female             0             0              12   United-States  
...        ...           ...           ...             ...             ...  
48773   Female             0             0              36   United-States  
48779     Male             0             0              40   United-States  
48789     Male             0             0              40   United-States  
48804   Female             0          1602              38   United-States  
48830   Female             0             0              40   United-States  

[2799 rows x 14 columns]

In [24]:
combined['race'].unique()

array([' White', ' Asian-Pac-Islander', ' Black', ' Amer-Indian-Eskimo',
       ' Other'], dtype=object)

In [25]:
combined[['race']].value_counts()

race               
 White                 41762
 Black                  4685
 Asian-Pac-Islander     1519
 Amer-Indian-Eskimo      470
 Other                   406
dtype: int64

### 인종

In [26]:
# one-hot-encoding
race_dummies = pd.get_dummies(combined['race'], prefix="race")
combined = pd.concat([combined, race_dummies],axis=1)
    
# 객실등급 특성 삭제
combined.drop('race',axis=1,inplace=True)

In [27]:
combined

age          workclass  fnlwgt      education  education-num  \
0       25            Private  219199           11th              7   
1       39            Private   52978   Some-college             10   
2       35            Private  196899      Bachelors             13   
3       64            Private  135527      Assoc-voc             11   
4       24            Private   60783   Some-college             10   
...    ...                ...     ...            ...            ...   
48837   45   Self-emp-not-inc  116789        HS-grad              9   
48838   48            Private  185079        HS-grad              9   
48839   63            Private  117473   Some-college             10   
48840   18            Private  150817           11th              7   
48841   31            Private  341632     Assoc-acdm             12   

            marital-status          occupation    relationship      sex  \
0                 Divorced   Machine-op-inspct   Not-in-family     Male   
1                 Divorced       Other-service   Not-in-family   Female   
2            Never-married   Handlers-cleaners   Not-in-family   Female   
3                 Divorced        Tech-support   Not-in-family   Female   
4       Married-civ-spouse    Transport-moving         Husband     Male   
...                    ...                 ...             ...      ...   
48837   Married-civ-spouse        Craft-repair         Husband     Male   
48838        Never-married     Exec-managerial   Not-in-family   Female   
48839   Married-civ-spouse      Prof-specialty         Husband     Male   
48840        Never-married               Sales       Own-child   Female   
48841   Married-civ-spouse        Tech-support         Husband     Male   

       capital-gain  capital-loss  hours-per-week  native-country  \
0                 0             0              40   United-States   
1                 0          1721              55   United-States   
2                 0             0              50           Haiti   
3                 0             0              40   United-States   
4                 0             0              70   United-States   
...             ...           ...             ...             ...   
48837             0             0              60   United-States   
48838             0             0              50   United-States   
48839          4386             0              40   United-States   
48840             0             0              20   United-States   
48841             0             0              46   United-States   

       race_ Amer-Indian-Eskimo  race_ Asian-Pac-Islander  race_ Black  \
0                             0                         0            0   
1                             0                         0            0   
2                             0                         1            0   
3                             0                         0            0   
4                             0                         0            0   
...                         ...                       ...          ...   
48837                         0                         0            0   
48838                         0                         0            0   
48839                         0                         0            0   
48840                         0                         0            0   
48841                         0                         0            1   

       race_ Other  race_ White  
0                0            1  
1                0            1  
2                0            0  
3                0            1  
4                0            1  
...            ...          ...  
48837            0            1  
48838            0            1  
48839            0            1  
48840            0            1  
48841            0            0  

[48842 rows x 18 columns]

In [28]:
combined[['workclass']].value_counts()

workclass        
 Private             36705
 Self-emp-not-inc     3862
 Local-gov            3136
 State-gov            1981
 Self-emp-inc         1695
 Federal-gov          1432
 Without-pay            21
 Never-worked           10
dtype: int64

### workclass

In [29]:
# one-hot-encoding
workclass_dummies = pd.get_dummies(combined['workclass'], prefix="workclass")
combined = pd.concat([combined, workclass_dummies],axis=1)
    
# 객실등급 특성 삭제
combined.drop('workclass',axis=1,inplace=True)

In [30]:
combined

age  fnlwgt      education  education-num       marital-status  \
0       25  219199           11th              7             Divorced   
1       39   52978   Some-college             10             Divorced   
2       35  196899      Bachelors             13        Never-married   
3       64  135527      Assoc-voc             11             Divorced   
4       24   60783   Some-college             10   Married-civ-spouse   
...    ...     ...            ...            ...                  ...   
48837   45  116789        HS-grad              9   Married-civ-spouse   
48838   48  185079        HS-grad              9        Never-married   
48839   63  117473   Some-college             10   Married-civ-spouse   
48840   18  150817           11th              7        Never-married   
48841   31  341632     Assoc-acdm             12   Married-civ-spouse   

               occupation    relationship      sex  capital-gain  \
0       Machine-op-inspct   Not-in-family     Male             0   
1           Other-service   Not-in-family   Female             0   
2       Handlers-cleaners   Not-in-family   Female             0   
3            Tech-support   Not-in-family   Female             0   
4        Transport-moving         Husband     Male             0   
...                   ...             ...      ...           ...   
48837        Craft-repair         Husband     Male             0   
48838     Exec-managerial   Not-in-family   Female             0   
48839      Prof-specialty         Husband     Male          4386   
48840               Sales       Own-child   Female             0   
48841        Tech-support         Husband     Male             0   

       capital-loss  ...  race_ Other race_ White  workclass_ Federal-gov  \
0                 0  ...            0           1                       0   
1              1721  ...            0           1                       0   
2                 0  ...            0           0                       0   
3                 0  ...            0           1                       0   
4                 0  ...            0           1                       0   
...             ...  ...          ...         ...                     ...   
48837             0  ...            0           1                       0   
48838             0  ...            0           1                       0   
48839             0  ...            0           1                       0   
48840             0  ...            0           1                       0   
48841             0  ...            0           0                       0   

       workclass_ Local-gov  workclass_ Never-worked  workclass_ Private  \
0                         0                        0                   1   
1                         0                        0                   1   
2                         0                        0                   1   
3                         0                        0                   1   
4                         0                        0                   1   
...                     ...                      ...                 ...   
48837                     0                        0                   0   
48838                     0                        0                   1   
48839                     0                        0                   1   
48840                     0                        0                   1   
48841                     0                        0                   1   

       workclass_ Self-emp-inc  workclass_ Self-emp-not-inc  \
0                            0                            0   
1                            0                            0   
2                            0                            0   
3                            0                            0   
4                            0                            0   
...                        ...                          ...   
48837                        0    

In [31]:
combined[['education']].value_counts()

education    
 HS-grad         15784
 Some-college    10878
 Bachelors        8025
 Masters          2657
 Assoc-voc        2061
 11th             1812
 Assoc-acdm       1601
 10th             1389
 7th-8th           955
 Prof-school       834
 9th               756
 12th              657
 Doctorate         594
 5th-6th           509
 1st-4th           247
 Preschool          83
dtype: int64

### education

In [32]:
# one-hot-encoding
education_dummies = pd.get_dummies(combined['education'], prefix="education")
combined = pd.concat([combined, education_dummies],axis=1)
    
# 객실등급 특성 삭제
combined.drop('education',axis=1,inplace=True)

In [33]:
combined.head(20)

age  fnlwgt  education-num          marital-status          occupation  \
0    25  219199              7                Divorced   Machine-op-inspct   
1    39   52978             10                Divorced       Other-service   
2    35  196899             13           Never-married   Handlers-cleaners   
3    64  135527             11                Divorced        Tech-support   
4    24   60783             10      Married-civ-spouse    Transport-moving   
5    66  274451              5      Married-civ-spouse     Farming-fishing   
6    56  152874             13                Divorced      Prof-specialty   
7    70  165586              9      Married-civ-spouse     Farming-fishing   
8    21  211013             11      Married-civ-spouse       Other-service   
9    33  192644              9               Separated   Handlers-cleaners   
10   45  174794             10                Divorced     Priv-house-serv   
11   54  231004             10                Divorced        Adm-clerical   
12   63  241404             10                Divorced     Protective-serv   
13   58   36091             14           Never-married     Exec-managerial   
14   44   53956             10      Married-civ-spouse     Farming-fishing   
15   31  162312             13           Never-married      Prof-specialty   
16   54  299324              3   Married-spouse-absent   Machine-op-inspct   
17   17   52012              7           Never-married       Other-service   
18   43  208613             15   Married-spouse-absent      Prof-specialty   
19   18  220168             10           Never-married                   O   

       relationship      sex  capital-gain  capital-loss  hours-per-week  ...  \
0     Not-in-family     Male             0             0              40  ...   
1     Not-in-family   Female             0          1721              55  ...   
2     Not-in-family   Female             0             0              50  ...   
3     Not-in-family   Female             0             0              40  ...   
4           Husband     Male             0             0              70  ...   
5           Husband     Male             0             0              25  ...   
6         Unmarried   Female             0          1741              40  ...   
7           Husband     Male             0             0              40  ...   
8    Other-relative   Female             0             0              50  ...   
9         Unmarried     Male             0             0              35  ...   
10    Not-in-family   Female             0             0              29  ...   
11    Not-in-family   Female             0             0              37  ...   
12    Not-in-family   Female             0             0              35  ...   
13    Not-in-family   Female             0             0              40  ...   
14          Husband     Male             0             0              45  ...   
15    Not-in-family     Male             0             0              35  ...   
16        Unmarried     Male             0             0              40  ...   
17        Own-child   Female             0             0              15  ...   
18    Not-in-family     Male         99999             0              40  ...   
19        Own-child     Male             0             0              16  ...   

   education_ 9th  education_ Assoc-acdm  education_ Assoc-voc  \
0               0                      0                     0   
1               0                      0                     0   
2               0                      0                     0   
3               0                      0                     1   
4               0                      0                     0   
5               1                      0                     0   
6               0                      0                     0   
7               0                      0                     0   
8               0                      0                     1   
9  

###  marital-status

In [34]:
combined[['marital-status']].value_counts()

marital-status        
 Married-civ-spouse       22379
 Never-married            16117
 Divorced                  6633
 Separated                 1530
 Widowed                   1518
 Married-spouse-absent      628
 Married-AF-spouse           37
dtype: int64

In [35]:
combined.corr()

age    fnlwgt  education-num  capital-gain  \
age                          1.000000 -0.076628       0.030940      0.077229   
fnlwgt                      -0.076628  1.000000      -0.038761     -0.003706   
education-num                0.030940 -0.038761       1.000000      0.125146   
capital-gain                 0.077229 -0.003706       0.125146      1.000000   
capital-loss                 0.056944 -0.004366       0.080972     -0.031441   
hours-per-week               0.071558 -0.013519       0.143689      0.082157   
race_ Amer-Indian-Eskimo    -0.014019 -0.064922      -0.026488     -0.007144   
race_ Asian-Pac-Islander    -0.010260 -0.050671       0.064153      0.011016   
race_ Black                 -0.017324  0.125910      -0.074364     -0.021456   
race_ Other                 -0.033295  0.005247      -0.044093     -0.001178   
race_ White                  0.032023 -0.063686       0.049287      0.014801   
workclass_ Federal-gov       0.049867 -0.009996       0.058072     -0.003633   
workclass_ Local-gov         0.057935  0.001233       0.097210     -0.009872   
workclass_ Never-worked     -0.019564  0.003438      -0.014350     -0.002072   
workclass_ Private          -0.189439  0.042687      -0.169104     -0.049687   
workclass_ Self-emp-inc      0.098962 -0.019165       0.079783      0.103144   
workclass_ Self-emp-not-inc  0.142956 -0.039082       0.017011      0.027630   
workclass_ State-gov         0.013029 -0.015051       0.104692     -0.008904   
workclass_ Without-pay       0.013073 -0.004274      -0.009081     -0.002098   
education_ 10th             -0.009244  0.011128      -0.271383     -0.017357   
education_ 11th             -0.093084  0.010108      -0.235006     -0.023056   
education_ 12th             -0.054573  0.008707      -0.094384     -0.013640   
education_ 1st-4th           0.041580  0.030621      -0.224010     -0.009141   
education_ 5th-6th           0.036990  0.039060      -0.282527     -0.009897   
education_ 7th-8th           0.112087 -0.002776      -0.333862     -0.015851   
education_ 9th               0.020655  0.011093      -0.247662     -0.012883   
education_ Assoc-acdm       -0.011636  0.007036       0.137618     -0.010922   
education_ Assoc-voc         0.000116 -0.020359       0.075266     -0.008463   
education_ Bachelors         0.008174 -0.005469       0.503937      0.040670   
education_ Doctorate         0.071919 -0.005856       0.255577      0.069217   
education_ HS-grad           0.021683 -0.006790      -0.289755     -0.046896   
education_ Masters           0.093825 -0.018588       0.365890      0.048426   
education_ Preschool         0.010717  0.019253      -0.145684     -0.001922   
education_ Prof-school       0.062571 -0.003842       0.252329      0.168158   
education_ Some-college     -0.115821  0.001726      -0.016259     -0.037289   

                             capital-loss  hours-per-week  \
age                              0.056944        0.071558   
fnlwgt                          -0.004366       -0.013519   
education-num                    0.080972        0.143689   
capital-gain                    -0.031441        0.082157   
capital-loss                     1.000000        0.054467   
hours-per-week                   0.054467        1.000000   
race_ Amer-Indian-Eskimo        -0.012157       -0.001194   
race_ Asian-Pac-Islander         0.003581       -0.007773   
race_ Black                     -0.020956       -0.047961   
race_ Other                     -0.005576       -0.009017   
race_ White                      0.020572        0.046606   
workclass_ Federal-gov           0.009221        0.015300   
workclass_ Local-gov             0.009504        0.008981   
workclass_ Never-worked         -0.003107       -0.013307   
workclass_ Private              -0.033380       -0.111492   
workclass_ Self-emp-inc          0.037036        0.124680   
workclass_ Self-emp-not-inc      0.015884        0.093944   
workclass_ State-gov            -0.002450       -0.0220

In [36]:
# one-hot-encoding
marital_status_dummies = pd.get_dummies(combined['marital-status'], prefix="marital-status")
combined = pd.concat([combined, marital_status_dummies],axis=1)
    
# 객실등급 특성 삭제
combined.drop('marital-status',axis=1,inplace=True)

In [37]:
combined

age  fnlwgt  education-num          occupation    relationship  \
0       25  219199              7   Machine-op-inspct   Not-in-family   
1       39   52978             10       Other-service   Not-in-family   
2       35  196899             13   Handlers-cleaners   Not-in-family   
3       64  135527             11        Tech-support   Not-in-family   
4       24   60783             10    Transport-moving         Husband   
...    ...     ...            ...                 ...             ...   
48837   45  116789              9        Craft-repair         Husband   
48838   48  185079              9     Exec-managerial   Not-in-family   
48839   63  117473             10      Prof-specialty         Husband   
48840   18  150817              7               Sales       Own-child   
48841   31  341632             12        Tech-support         Husband   

           sex  capital-gain  capital-loss  hours-per-week  native-country  \
0         Male             0             0              40   United-States   
1       Female             0          1721              55   United-States   
2       Female             0             0              50           Haiti   
3       Female             0             0              40   United-States   
4         Male             0             0              70   United-States   
...        ...           ...           ...             ...             ...   
48837     Male             0             0              60   United-States   
48838   Female             0             0              50   United-States   
48839     Male          4386             0              40   United-States   
48840   Female             0             0              20   United-States   
48841     Male             0             0              46   United-States   

       ...  education_ Preschool  education_ Prof-school  \
0      ...                     0                       0   
1      ...                     0                       0   
2      ...                     0                       0   
3      ...                     0                       0   
4      ...                     0                       0   
...    ...                   ...                     ...   
48837  ...                     0                       0   
48838  ...                     0                       0   
48839  ...                     0                       0   
48840  ...                     0                       0   
48841  ...                     0                       0   

       education_ Some-college  marital-status_ Divorced  \
0                            0                         1   
1                            1                         1   
2                            0                         0   
3                            0                         1   
4                            1                         0   
...                        ...                       ...   
48837                        0                         0   
48838                        0                         0   
48839                        1                         0   
48840                        0                         0   
48841                        0                         0   

       marital-status_ Married-AF-spouse  marital-status_ Married-civ-spouse  \
0                                      0                                   0   
1                                      0                                   0   
2                                      0                                   0   
3                                      0                                   0   
4                                      0                                   1   
...                                  ...                                 ...   
48837                                  0                                   1   
48838                                  0                                   0   
48839                          

In [38]:
combined[['occupation']].value_counts()

occupation        
 Prof-specialty       6172
 Craft-repair         6112
 Exec-managerial      6086
 Adm-clerical         5611
 Sales                5504
 Other-service        4923
 Machine-op-inspct    3022
O                     2809
 Transport-moving     2355
 Handlers-cleaners    2072
 Farming-fishing      1490
 Tech-support         1446
 Protective-serv       983
 Priv-house-serv       242
 Armed-Forces           15
dtype: int64

In [39]:
# one-hot-encoding
occupation_status_dummies = pd.get_dummies(combined['occupation'], prefix="occupation")
combined = pd.concat([combined, occupation_status_dummies],axis=1)
    
# 객실등급 특성 삭제
combined.drop('occupation',axis=1,inplace=True)

In [40]:
combined

age  fnlwgt  education-num    relationship      sex  capital-gain  \
0       25  219199              7   Not-in-family     Male             0   
1       39   52978             10   Not-in-family   Female             0   
2       35  196899             13   Not-in-family   Female             0   
3       64  135527             11   Not-in-family   Female             0   
4       24   60783             10         Husband     Male             0   
...    ...     ...            ...             ...      ...           ...   
48837   45  116789              9         Husband     Male             0   
48838   48  185079              9   Not-in-family   Female             0   
48839   63  117473             10         Husband     Male          4386   
48840   18  150817              7       Own-child   Female             0   
48841   31  341632             12         Husband     Male             0   

       capital-loss  hours-per-week  native-country  race_ Amer-Indian-Eskimo  \
0                 0              40   United-States                         0   
1              1721              55   United-States                         0   
2                 0              50           Haiti                         0   
3                 0              40   United-States                         0   
4                 0              70   United-States                         0   
...             ...             ...             ...                       ...   
48837             0              60   United-States                         0   
48838             0              50   United-States                         0   
48839             0              40   United-States                         0   
48840             0              20   United-States                         0   
48841             0              46   United-States                         0   

       ...  occupation_ Handlers-cleaners  occupation_ Machine-op-inspct  \
0      ...                              0                              1   
1      ...                              0                              0   
2      ...                              1                              0   
3      ...                              0                              0   
4      ...                              0                              0   
...    ...                            ...                            ...   
48837  ...                              0                              0   
48838  ...                              0                              0   
48839  ...                              0                              0   
48840  ...                              0                              0   
48841  ...                              0                              0   

       occupation_ Other-service  occupation_ Priv-house-serv  \
0                              0                            0   
1                              1                            0   
2                              0                            0   
3                              0                            0   
4                              0                            0   
...                          ...                          ...   
48837                          0                            0   
48838                          0                            0   
48839                          0                            0   
48840                          0                            0   
48841                          0                            0   

       occupation_ Prof-specialty  occupation_ Protective-serv  \
0                               0                            0   
1                               0                            0   
2                               0                            0   
3                               0                            0   
4                               0                            0   
...                           .

In [41]:
combined[['relationship']].value_counts()

relationship   
 Husband           19716
 Not-in-family     12583
 Own-child          7581
 Unmarried          5125
 Wife               2331
 Other-relative     1506
dtype: int64

In [42]:
# one-hot-encoding
relationship_status_dummies = pd.get_dummies(combined['relationship'], prefix="relationship")
combined = pd.concat([combined, relationship_status_dummies],axis=1)
    
# 객실등급 특성 삭제
combined.drop('relationship',axis=1,inplace=True)

In [43]:
combined

age  fnlwgt  education-num      sex  capital-gain  capital-loss  \
0       25  219199              7     Male             0             0   
1       39   52978             10   Female             0          1721   
2       35  196899             13   Female             0             0   
3       64  135527             11   Female             0             0   
4       24   60783             10     Male             0             0   
...    ...     ...            ...      ...           ...           ...   
48837   45  116789              9     Male             0             0   
48838   48  185079              9   Female             0             0   
48839   63  117473             10     Male          4386             0   
48840   18  150817              7   Female             0             0   
48841   31  341632             12     Male             0             0   

       hours-per-week  native-country  race_ Amer-Indian-Eskimo  \
0                  40   United-States                         0   
1                  55   United-States                         0   
2                  50           Haiti                         0   
3                  40   United-States                         0   
4                  70   United-States                         0   
...               ...             ...                       ...   
48837              60   United-States                         0   
48838              50   United-States                         0   
48839              40   United-States                         0   
48840              20   United-States                         0   
48841              46   United-States                         0   

       race_ Asian-Pac-Islander  ...  occupation_ Sales  \
0                             0  ...                  0   
1                             0  ...                  0   
2                             1  ...                  0   
3                             0  ...                  0   
4                             0  ...                  0   
...                         ...  ...                ...   
48837                         0  ...                  0   
48838                         0  ...                  0   
48839                         0  ...                  0   
48840                         0  ...                  1   
48841                         0  ...                  0   

       occupation_ Tech-support  occupation_ Transport-moving  occupation_O  \
0                             0                             0             0   
1                             0                             0             0   
2                             0                             0             0   
3                             1                             0             0   
4                             0                             1             0   
...                         ...                           ...           ...   
48837                         0                             0             0   
48838                         0                             0             0   
48839                         0                             0             0   
48840                         0                             0             0   
48841                         1                             0             0   

       relationship_ Husband  relationship_ Not-in-family  \
0                          0                            1   
1                          0                            1   
2                          0                            1   
3                          0                            1   
4                          1                            0   
...                      ...                          ...   
48837                      1                            0   
48838                      0                            1   
48839                      1                            0   
48840                      0                 

In [44]:
combined[['native-country']].value_counts()

native-country             
 United-States                 44689
 Mexico                          951
 Philippines                     295
 Germany                         206
 Puerto-Rico                     184
 Canada                          182
 El-Salvador                     155
 India                           151
 Cuba                            138
 England                         127
 China                           122
 South                           115
 Jamaica                         106
 Italy                           105
 Dominican-Republic              103
 Japan                            92
 Guatemala                        88
 Poland                           87
 Vietnam                          86
 Columbia                         85
 Haiti                            75
 Portugal                         67
 Taiwan                           65
 Iran                             59
 Greece                           49
 Nicaragua                        49
 Peru     

In [45]:
# one-hot-encoding
native_country_status_dummies = pd.get_dummies(combined['native-country'], prefix="native-country")
combined = pd.concat([combined, native_country_status_dummies],axis=1)
    
# 객실등급 특성 삭제
combined.drop('native-country',axis=1,inplace=True)

In [46]:
combined

age  fnlwgt  education-num      sex  capital-gain  capital-loss  \
0       25  219199              7     Male             0             0   
1       39   52978             10   Female             0          1721   
2       35  196899             13   Female             0             0   
3       64  135527             11   Female             0             0   
4       24   60783             10     Male             0             0   
...    ...     ...            ...      ...           ...           ...   
48837   45  116789              9     Male             0             0   
48838   48  185079              9   Female             0             0   
48839   63  117473             10     Male          4386             0   
48840   18  150817              7   Female             0             0   
48841   31  341632             12     Male             0             0   

       hours-per-week  race_ Amer-Indian-Eskimo  race_ Asian-Pac-Islander  \
0                  40                         0                         0   
1                  55                         0                         0   
2                  50                         0                         1   
3                  40                         0                         0   
4                  70                         0                         0   
...               ...                       ...                       ...   
48837              60                         0                         0   
48838              50                         0                         0   
48839              40                         0                         0   
48840              20                         0                         0   
48841              46                         0                         0   

       race_ Black  ...  native-country_ Portugal  \
0                0  ...                         0   
1                0  ...                         0   
2                0  ...                         0   
3                0  ...                         0   
4                0  ...                         0   
...            ...  ...                       ...   
48837            0  ...                         0   
48838            0  ...                         0   
48839            0  ...                         0   
48840            0  ...                         0   
48841            1  ...                         0   

       native-country_ Puerto-Rico  native-country_ Scotland  \
0                                0                         0   
1                                0                         0   
2                                0                         0   
3                                0                         0   
4                                0                         0   
...                            ...                       ...   
48837                            0                         0   
48838                            0                         0   
48839                            0                         0   
48840                            0                         0   
48841                            0                         0   

       native-country_ South  native-country_ Taiwan  \
0                          0                       0   
1                          0                       0   
2                          0                       0   
3                          0                       0   
4                          0                       0   
...                      ...                     ...   
48837                      0                       0   
48838                      0                       0   
48839                      0                       0   
48840                      0                       0   
48841                      0                       0   

       native-country_ Thailand  native-country_ Trinadad&Tobago  \
0                             0               

## 성별 처리

In [47]:
combined['sex'] = combined['sex'].map({' Male':0, ' Female':1})

In [48]:
combined

age  fnlwgt  education-num  sex  capital-gain  capital-loss  \
0       25  219199              7    0             0             0   
1       39   52978             10    1             0          1721   
2       35  196899             13    1             0             0   
3       64  135527             11    1             0             0   
4       24   60783             10    0             0             0   
...    ...     ...            ...  ...           ...           ...   
48837   45  116789              9    0             0             0   
48838   48  185079              9    1             0             0   
48839   63  117473             10    0          4386             0   
48840   18  150817              7    1             0             0   
48841   31  341632             12    0             0             0   

       hours-per-week  race_ Amer-Indian-Eskimo  race_ Asian-Pac-Islander  \
0                  40                         0                         0   
1                  55                         0                         0   
2                  50                         0                         1   
3                  40                         0                         0   
4                  70                         0                         0   
...               ...                       ...                       ...   
48837              60                         0                         0   
48838              50                         0                         0   
48839              40                         0                         0   
48840              20                         0                         0   
48841              46                         0                         0   

       race_ Black  ...  native-country_ Portugal  \
0                0  ...                         0   
1                0  ...                         0   
2                0  ...                         0   
3                0  ...                         0   
4                0  ...                         0   
...            ...  ...                       ...   
48837            0  ...                         0   
48838            0  ...                         0   
48839            0  ...                         0   
48840            0  ...                         0   
48841            1  ...                         0   

       native-country_ Puerto-Rico  native-country_ Scotland  \
0                                0                         0   
1                                0                         0   
2                                0                         0   
3                                0                         0   
4                                0                         0   
...                            ...                       ...   
48837                            0                         0   
48838                            0                         0   
48839                            0                         0   
48840                            0                         0   
48841                            0                         0   

       native-country_ South  native-country_ Taiwan  \
0                          0                       0   
1                          0                       0   
2                          0                       0   
3                          0                       0   
4                          0                       0   
...                      ...                     ...   
48837                      0                       0   
48838                      0                       0   
48839                      0                       0   
48840                      0                       0   
48841                      0                       0   

       native-country_ Thailand  native-country_ Trinadad&Tobago  \
0                             0                                0   
1                         

In [49]:
combined.head(15)

age  fnlwgt  education-num  sex  capital-gain  capital-loss  \
0    25  219199              7    0             0             0   
1    39   52978             10    1             0          1721   
2    35  196899             13    1             0             0   
3    64  135527             11    1             0             0   
4    24   60783             10    0             0             0   
5    66  274451              5    0             0             0   
6    56  152874             13    1             0          1741   
7    70  165586              9    0             0             0   
8    21  211013             11    1             0             0   
9    33  192644              9    0             0             0   
10   45  174794             10    1             0             0   
11   54  231004             10    1             0             0   
12   63  241404             10    1             0             0   
13   58   36091             14    1             0             0   
14   44   53956             10    0             0             0   

    hours-per-week  race_ Amer-Indian-Eskimo  race_ Asian-Pac-Islander  \
0               40                         0                         0   
1               55                         0                         0   
2               50                         0                         1   
3               40                         0                         0   
4               70                         0                         0   
5               25                         0                         0   
6               40                         0                         0   
7               40                         0                         0   
8               50                         0                         0   
9               35                         0                         0   
10              29                         0                         0   
11              37                         0                         0   
12              35                         0                         0   
13              40                         1                         0   
14              45                         0                         0   

    race_ Black  ...  native-country_ Portugal  native-country_ Puerto-Rico  \
0             0  ...                         0                            0   
1             0  ...                         0                            0   
2             0  ...                         0                            0   
3             0  ...                         0                            0   
4             0  ...                         0                            0   
5             0  ...                         0                            0   
6             1  ...                         0                            0   
7             0  ...                         0                            0   
8             0  ...                         0                            0   
9             0  ...                         0                            1   
10            0  ...                         0                            0   
11            0  ...                         0                            0   
12            0  ...                         0                            0   
13            0  ...                         0                            0   
14            0  ...                         0                            0   

    native-country_ Scotland  native-country_ South  native-country_ Taiwan  \
0                          0                      0                       0   
1                          0                      0                       0   
2                          0                      0                       0   
3                          0                      0                       0   
4                          0                      0                       0   
5      

In [50]:
X_train = combined.iloc[:29305]
y_train = targets
X_test = combined.iloc[29305:] # 행만... 뒤에 , 붙이고 쓰면 행,열이 된다

In [51]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)

(29305, 105)
(29305,)
(19537, 105)


# 앙상블~~~~!!!

In [52]:
# 다른 분류 모델들과 같이 살펴보자 
from sklearn.ensemble import VotingClassifier # 처음 배움, 서로 다른 모델을 앙상블 하는 것
from sklearn.ensemble import RandomForestClassifier # 각각의 tree모델 앙상블
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression # 선형 분류 모델
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import LinearSVC # 선형 분류 모델, 서포트 벡터 머신

In [53]:
knn_model = KNeighborsClassifier()
tree_model = DecisionTreeClassifier()
forest_model = RandomForestClassifier()

In [257]:
# voting_model = VotingClassifier(estimators = [('knn1', knn_model), ('tree1', tree_model), ('forest1', forest_model)], voting = 'soft') 

In [256]:
voting_model

NameError: name 'voting_model' is not defined

In [ ]:
voting_model.fit(X_train, y_train)

In [ ]:
voting_model.predict(X_test)

In [ ]:
pre = voting_model.predict(X_test)  # 이걸 저장해서 캐글에 올려보면 정확도 확인 가능
pre

In [ ]:
result = pd.read_csv('data/sample_submission.csv')
result['income'] = pre

In [ ]:
result.to_csv('H_submission_01.csv', index = False)    # 0.84463

In [ ]:
forest_model2 = RandomForestClassifier(n_estimators = 1200, max_features = 0.4, 
                                       max_depth = 5, min_samples_leaf = 5, max_leaf_nodes = 50)

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
param_grid = {
    'max_depth' : [3,5,10],
    'n_estimators' : [700,900,1200],
    'max_features' : [0.3, 0.5, 0.7],
    'max_leaf_nodes' : [15,35,70]
}


In [ ]:
grid = GridSearchCV(forest_model2, param_grid, cv=7, verbose = 1)

In [ ]:
grid

In [ ]:
grid.fit(X_train, y_train)

In [ ]:
print(grid.best_score_) 
print(grid.best_params_) 

In [ ]:
final_forest_model = RandomForestClassifier(max_depth = 15, max_features = 0.7, max_leaf_nodes = 80, n_estimators = 1500)

In [ ]:
final_forest_model.fit(X_train, y_train)

In [ ]:
final_pre = final_forest_model.predict(X_test)

In [ ]:
result2 = pd.read_csv('data/sample_submission.csv')
result2['income'] = final_pre

In [ ]:
result2.to_csv('H_submission_02.csv', index = False) # 0.87240

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, random_state = 7)

In [ ]:
combined.corr()

In [ ]:
from sklearn.metrics import accuracy_score 

In [ ]:
!pip install xgboost

from xgboost import XGBClassifier

In [ ]:
print(X_train.shape)
print(y_train.shape)

In [ ]:
xg_model = XGBClassifier(n_estimators = 1700, learning_rate = 0.01, max_depth = 7)

In [ ]:
xg_model.fit(X_train,y_train)

In [ ]:
pre = xg_model.predict(X_test)
pre

In [ ]:
result2 = pd.read_csv('data/sample_submission.csv')
result2['income'] = pre

In [ ]:
result2.to_csv('H_submission043.csv', index = False) # 0.88 = 1500/0.01/7    # 1250/0.01/4  = 0.879    #1700/0.07/7  = 0.880

In [ ]:
xg_model.score(X_train, y_train)

In [54]:
from xgboost import XGBClassifier
xg_model = XGBClassifier(n_estimators = 1500, learning_rate = 0.02, max_depth = 7)

In [55]:
param_grid2 = {
    'max_depth' : [3,5,7,10],
    'n_estimators' : [700,1000,1500,1800],
    'learning_rate' : [0.01,0.05],
}

In [56]:
from sklearn.model_selection import GridSearchCV
grid2 = GridSearchCV(xg_model,param_grid2, cv = 7, verbose = 3)


In [57]:
grid2.fit(X_train,y_train)

Fitting 7 folds for each of 32 candidates, totalling 224 fits
[CV] learning_rate=0.01, max_depth=3, n_estimators=700 ...............
[14:43:39] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
C:\ProgramData\Anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[CV]  learning_rate=0.01, max_depth=3, n_estimators=700, score=0.862, total=   4.5s
[CV] learning_rate=0.01, max_depth=3, n_estimators=700 ...............
[14:43:43] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    4.4s remaining:    0.0s


[CV]  learning_rate=0.01, max_depth=3, n_estimators=700, score=0.862, total=   4.4s
[CV] learning_rate=0.01, max_depth=3, n_estimators=700 ...............
[14:43:48] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


...

[CV]  learning_rate=0.05, max_depth=7, n_estimators=1500, score=0.868, total=  18.5s
[CV] learning_rate=0.05, max_depth=7, n_estimators=1500 ..............
[15:22:13] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[CV]  learning_rate=0.05, max_depth=7, n_estimators=1500, score=0.865, total=  18.2s
[CV] learning_rate=0.05, max_depth=7, n_estimators=1500 ..............
[15:22:31] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[CV]  learning_rate=0.05, max_depth=7, n_estimators=1500, score=0.87

[CV]  learning_rate=0.05, max_depth=10, n_estimators=700, score=0.864, total=  12.1s
[CV] learning_rate=0.05, max_depth=10, n_estimators=700 ..............
[15:27:38] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[CV]  learning_rate=0.05, max_depth=10, n_estimators=700, score=0.869, total=  12.9s
[CV] learning_rate=0.05, max_depth=10, n_estimators=700 ..............
[15:27:51] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[CV]  learning_rate=0.05, max_depth=10, n_estimators=700, score=0.86

[CV]  learning_rate=0.05, max_depth=10, n_estimators=1800, score=0.859, total=  31.0s
[CV] learning_rate=0.05, max_depth=10, n_estimators=1800 .............
[15:34:10] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[CV]  learning_rate=0.05, max_depth=10, n_estimators=1800, score=0.871, total=  30.9s
[CV] learning_rate=0.05, max_depth=10, n_estimators=1800 .............
[15:34:40] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[CV]  learning_rate=0.05, max_depth=10, n_estimators=1800, score=0

[Parallel(n_jobs=1)]: Done 224 out of 224 | elapsed: 53.1min finished


GridSearchCV(cv=7,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, gamma=None,
                                     gpu_id=None, importance_type='gain',
                                     interaction_constraints=None,
                                     learning_rate=0.01, max_delta_step=None,
                                     max_depth=7, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     n_estimators=1500, n_jobs=None,
                                     num_parallel_tree=None, random_state=None,
                                     reg_alpha=None, reg_lambda=None,
                                     scale_pos_weight=None, subsample=None,
                                     tree_method=None, v

In [58]:
print(grid2.best_score_) 
print(grid2.best_params_) 

0.8734344876967636
{'learning_rate': 0.01, 'max_depth': 5, 'n_estimators': 1800}


In [77]:
xg_model = XGBClassifier(max_depth = 7, n_estimators = 2500, learning_rate = 0.01)
xg_model.fit(X_train,y_train)
xg_model.score(X_train,y_train)

[15:50:13] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


0.9023716089404539

In [78]:
pre = xg_model.predict(X_test)
pre

array([0, 0, 0, ..., 1, 0, 0], dtype=int64)

In [79]:
result2 = pd.read_csv('data/sample_submission.csv')
result2['income'] = pre

In [80]:
result2.to_csv('US_submission_03.csv', index = False) # 0.88   # 0.87  
# mec - 0.881 
# phil01 - 0.879 5 1800 0.01 
# phil02 - 0.879 5 1600 0.01
#phil03 - 0.879  7 1800 0.01
# US01 -  0.880 - 5 1800 0.01
# US02 - 0.87944 - 7 1800 0.01
# us03 - 0.877 - 7 2500 0.01  과대적합

In [ ]:
!pip install catboost

In [ ]:
#cat_model = CatBoostRegressor(iterations=1000, learning_rate=0.01,depth=7)

In [ ]:
#cat_model.fit(X_train, y_train)

In [ ]:
#print(cat_model.best_score_) 
#print(cat_model.best_params_) 

In [ ]:
#preds = cat_model.predict(X_test)
#preds

In [ ]:
#result3 = pd.read_csv('data/sample_submission.csv')
#result3['income'] = preds

In [ ]:
#result3.to_csv('H_submission05.csv', index = False)

In [ ]:
from sklearn.metrics import accuracy_score 

In [ ]:
cat_model = CatBoostClassifier(learning_rate = 0.02)
cat_model.fit(X_train, y_train)
preds = cat_model.predict(X_test)

print('CatBoost results:')


In [ ]:
print(cat_model.best_score_) 

In [ ]:
from catboost import CatBoostClassifier
cat_model = CatBoostClassifier(learning_rate = 0.02)
cat_model.fit(X_train,y_train)
cat_model.score(X_train,y_train)


In [ ]:
pre1 = cat_model.predict(X_test)
pre1

In [ ]:
result3 = pd.read_csv('data/sample_submission.csv')
result3['income'] = pre1

In [ ]:
result3.to_csv('H_submission12.csv', index = False)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, random_state = 7)
logreg = LogisticRegression(max_iter = 1000)

In [ ]:
X_train

In [ ]:
logreg.fit(X_train,y_train)
print("test 점수 : {:.2f}".format(logreg.score(X_test, y_test)))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, random_state = 7)
logreg = LogisticRegression(max_iter = 1000)
logreg.fit(X_train,y_train)
print("test 점수 : {:.2f}".format(logreg.score(X_test, y_test)))

In [ ]:
!pip install bayesian-optimization

In [ ]:
from sklearn.model_selection import cross_val_score
from bayes_opt import BayesianOptimization
import xgboost as xgb

In [ ]:
def XGB_cv(max_depth,learning_rate, n_estimators, gamma
           ,min_child_weight, max_delta_step, subsample
           ,colsample_bytree, silent=True, nthread=-1):
    model = xgb.XGBClassifier(max_depth=int(max_depth),
                              learning_rate=learning_rate,
                              n_estimators=int(n_estimators),
                              silent=silent,
                              nthread=nthread,
                              gamma=gamma,
                              min_child_weight=min_child_weight,
                              max_delta_step=max_delta_step,
                              subsample=subsample,
                              colsample_bytree=colsample_bytree)
    RMSE = cross_val_score(model, X_train, y_train, scoring='accuracy', cv=7).mean()
    return -RMSE

# 주어진 범위 사이에서 적절한 값을 찾는다.
pbounds = {'max_depth': (5, 15),
          'learning_rate': (0.01, 0.5),
          'n_estimators': (50, 1600),
          'gamma': (0.01, 0.05),
          'min_child_weight': (1, 15),
          'max_delta_step': (0, 0.2),
          'subsample': (0.1, 0.7),
          'colsample_bytree' :(0.1, 0.99)
          }

In [ ]:
xgboostBO = BayesianOptimization(f = XGB_cv,pbounds = pbounds, verbose = 2, random_state = 77)

In [ ]:
# 메소드를 이용해 최대화!
xgboostBO.maximize(init_points=2, n_iter = 700, acq='ei', xi=0.01)


In [ ]:

xgboostBO.max# 찾은 파라미터 값 확인

In [ ]:
fit_xgb = xgb.XGBClassifier(max_depth= int( xgboostBO.max['params']['max_depth'] ),
                             learning_rate=xgboostBO.max['params']['learning_rate'],
                             n_estimators=int(xgboostBO.max['params']['n_estimators']),
                             gamma= xgboostBO.max['params']['gamma'],
                             min_child_weight=xgboostBO.max['params']['min_child_weight'],
                             max_delta_step=xgboostBO.max['params']['max_delta_step'],
                             subsample=xgboostBO.max['params']['subsample'],
                             colsample_bytree=xgboostBO.max['params']['colsample_bytree'])

In [ ]:
model1  = fit_xgb.fit(X_train,y_train)
model1

In [ ]:

pre1 = fit_xgb.predict(X_test)
pre1

In [ ]:

result1 = pd.read_csv('data/sample_submission.csv')
result1['income'] = pre1

result1.to_csv('H_submission_060.csv', index = False) # 0.88  # 0.87  # 0.87  ##0.77  ##

# 랜덤 서치!

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
model = XGBClassifier(n_estimators = 1500, learning_rate = 0.01, max_depth = 7)

In [ ]:
print(model.get_params().keys())


In [ ]:
param_distributions = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [None, 2, 3, 4, 5, 6,7,10],
    'n_estimators' : [700,1000,1500,1800],
    'learning_rate' : [0.01,0.05,0.1]
}

In [ ]:
randomized_search = RandomizedSearchCV(model, param_distributions=param_distributions, n_iter=10, 
                                       return_train_score=True) 


In [ ]:
randomized_search = RandomizedSearchCV(model, param_distributions=param_distributions, n_iter=20,
                                       cv=5, scoring='accuracy', return_train_score=True, verbose = 3, random_state = 1) 

In [ ]:
randomized_search.fit(X_train, y_train)


In [ ]:
df = pd.DataFrame(randomized_search.cv_results_).sort_values(by=['mean_test_score', 'mean_train_score'], ascending=False)
print(df[['params', 'mean_train_score', 'mean_test_score']]) 

In [ ]:
print(randomized_search.best_score_) 
print(randomized_search.best_params_) 

In [ ]:
pre2 = randomized_search.best_estimator_.predict(X_test)
pre2

In [ ]:

result1 = pd.read_csv('data/sample_submission.csv')
result1['income'] = pre2

result1.to_csv('H_submission_057.csv', index = False) # 0.88  # 0.87  # 0.87  #0.77  # 0.87